In [ ]:
!pip install odfpy
import pandas as pd
import re
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=deac965c370f17076bd0ecd2e36585ba1655836c7e3581dac6db78cf29b822a1
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy


In [ ]:
import pandas as pd
import os

# List of file base names
file_base_names = [
    "Brighton_and_Hove_East_Sussex_West_Sussex_LSIP_Main_Tables",
    "Buckinghamshire_LSIP_Main_Tables",
    "Cambridgeshire_and_Peterborough_LSIP_Main_Tables",
    "Cheshire_and_Warrington_LSIP_Main_Tables",
    "Cornwall_and_the_Isles_of_Scilly_LSIP_Main_Tables",
    "Cumbria_LSIP_Main_Tables",
    "Derbyshire_and_Nottinghamshire_LSIP_Main_Tables",
    "Dorset_LSIP_Main_Tables",
    "Essex_Southend-on-Sea_and_Thurrock_LSIP_Main_Tables",
    "Gloucestershire_LSIP_Main_Tables",
    "Greater_Lincolnshire_LSIP_Main_Tables",
    "Greater_London_LSIP_Main_Tables",
    "Greater_Manchester_LSIP_Main_Tables",
    "Heart_of_the_South-West_LSIP_Main_Tables",
    "Hertfordshire_LSIP_Main_Tables",
    "Hull_and_East_Yorkshire_LSIP_Main_Tables",
    "Kent_and_Medway_LSIP_Main_Tables",
    "Lancashire_LSIP_Main_Tables",
    "Leicester_and_Leicestershire_LSIP_Main_Tables",
    "Liverpool_City_Region_LSIP_Main_Tables",
    "Norfolk_and_Suffolk_LSIP_Main_Tables",
    "North_East_LSIP_Main_Tables",
    "North_of_Tyne_LSIP_Main_Tables",
    "Oxfordshire_LSIP_Main_Tables",
    "Solent_LSIP_Main_Tables",
    "South_East_Midlands_LSIP_Main_Tables",
    "South_Yorkshire_LSIP_Main_Tables",
    "Stoke-on-Trent_and_Staffordshire_LSIP_Main_Tables",
    "Swindon_and_Wiltshire_LSIP_Main_Tables",
    "Tees_Valley_LSIP_Main_Tables",
    "Thames_Valley_Berkshire_LSIP_Main_Tables",
    "The_Marches_LSIP_Main_Tables",
    "West_Midlands_and_Warwickshire_LSIP_Main_Tables",
    "West_of_England_and_North_Somerset_LSIP_Main_Tables",
    "West_Yorkshire_LSIP_Main_Tables",
    "Worcestershire_LSIP_Main_Tables",
    "York_and_North_Yorkshire_LSIP_Main_Tables"
]

# Empty list to store processed dataframes
all_regions_data = []

# Loop through all file names
for base_name in file_base_names:
    file_path = f"{base_name}.ods"

    try:
        # Load the "Qual_T1" sheet
        qual_t1_df = pd.read_excel(file_path, sheet_name="Qual_T1", header=None)

        # Process "Levels (000s)"
        iolc_df = qual_t1_df.iloc[2:13, 0:6]  # Extract specific range
        iolc_df_cleaned = iolc_df.dropna(how='all').reset_index(drop=True)  # Remove empty rows
        iolc_df_transposed = iolc_df_cleaned.T
        iolc_df_transposed.columns = iolc_df_transposed.iloc[0]  # Set first row as header
        iolc_df_transposed = iolc_df_transposed[1:].reset_index(drop=True)

        # Ensure "Year" column is clean
        if 'Year' not in iolc_df_transposed.columns:
            iolc_df_transposed.insert(0, 'Year', iolc_df_transposed.iloc[:, 0])  # Use first column as Year
            iolc_df_transposed.drop(iolc_df_transposed.columns[1], axis=1, inplace=True)  # Remove duplicate column

        # Extract the region name from filename
        region = base_name.split("_LSIP_Main_Tables")[0]
        iolc_df_transposed["Region"] = region
        iolc_df_transposed["Type"] = "Levels (000s)"

        # Reorder columns
        column_order = ["Region", "Type", "Year"] + [col for col in iolc_df_transposed.columns if col not in ["Region", "Type", "Year"]]
        iolc_df_reordered = iolc_df_transposed[column_order]

        #Process "Shares (per cent)"
        iolc_df_new = qual_t1_df.iloc[17:27, 0:6]  # Extract another specific range
        iolc_df_new.dropna(how='all', inplace=True)  # Remove empty rows
        iolc_df_new.iloc[0, 0] = "Year"  # Rename first column
        iolc_df_transposed_new = iolc_df_new.T
        iolc_df_transposed_new.columns = iolc_df_transposed_new.iloc[0]  # Set first row as header
        iolc_df_transposed_new = iolc_df_transposed_new[1:].reset_index(drop=True)

        # Ensure "Year" column is clean
        if 'Year' not in iolc_df_transposed_new.columns:
            iolc_df_transposed_new.insert(0, 'Year', iolc_df_transposed_new.iloc[:, 0])
            iolc_df_transposed_new.drop(iolc_df_transposed_new.columns[1], axis=1, inplace=True)

        iolc_df_transposed_new["Region"] = region
        iolc_df_transposed_new["Type"] = "Shares (per cent)"

        # Reorder columns
        column_order = ["Region", "Type", "Year"] + [col for col in iolc_df_transposed_new.columns if col not in ["Region", "Type", "Year"]]
        iolc_df_reordered_new = iolc_df_transposed_new[column_order]

        # Combine datasets
        iolc_combined_df = pd.concat([iolc_df_reordered, iolc_df_reordered_new], ignore_index=True)
        all_regions_data.append(iolc_combined_df)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Combine all regions into a single dataframe
final_df = pd.concat(all_regions_data, ignore_index=True)

# Remove any NaN values in "Year" column
final_df = final_df.dropna(subset=["Year"])

# Convert "Year" column to string (ensuring consistency)
final_df["Year"] = final_df["Year"].astype(str).str.strip()


In [ ]:
final_LSIDs_df = final_df.drop("Levels (000s)", axis=1)
final_LSIDs_df


,Region,Type,Year,RQF8 Doctorate,RQF7 Other higher degree,RQF6 First degree,RQF5 Foundation degree;Nursing;Teaching,RQF4 HE below degree level,RQF3 A level & equivalent,RQF2 GCSE(A-C) & equivalent,RQF1 GCSE(below grade C) & equivalent,No Qualification
0,Brighton_and_Hove_East_Sussex_West_Sussex,Levels (000s),2015,10.759075,77.352391,166.683622,50.521118,46.834203,172.463214,167.15644,112.903705,31.959232
1,Brighton_and_Hove_East_Sussex_West_Sussex,Levels (000s),2019,13.634878,98.149724,190.160919,53.977809,53.223037,169.586958,167.212827,103.988125,32.863723
2,Brighton_and_Hove_East_Sussex_West_Sussex,Levels (000s),2020,14.663621,105.428567,188.832573,53.348658,53.107942,176.050109,152.397921,89.725107,28.336502
3,Brighton_and_Hove_East_Sussex_West_Sussex,Levels (000s),2025,16.934407,121.773724,201.90157,53.095422,54.503634,172.25138,148.184285,83.795899,26.62368
4,Brighton_and_Hove_East_Sussex_West_Sussex,Levels (000s),2035,21.456845,154.271721,232.445602,58.686858,62.0451,171.364668,125.028013,55.963008,25.247186
...,...,...,...,...,...,...,...,...,...,...,...,...
365,York_and_North_Yorkshire,Shares (per cent),2015,1.122701,7.568247,16.049276,5.22427,4.700141,21.642767,21.539632,15.855275,6.297691
366,York_and_North_Yorkshire,Shares (per cent),2019,1.317718,9.223968,17.530927,5.075707,4.958217,21.925661,20.085859,14.103054,5.77889
367,York_and_North_Yorkshire,Shares (per cent),2020,1.257903,8.805639,17.883253,4.952294,4.96684,23.491925,21.489234,12.65934,4.493572
368,York_and_North_Yorkshire,Shares (per cent),2025,1.684961,11.928834,19.866702,5.196988,5.479711,22.549728,18.441264,11.238938,3.612875


In [ ]:
# Save the DataFrame as an Excel file
final_LSIDs_df.to_excel("qual_T1_.xlsx", index=False)